# 2. Get Extended data from BoM Water Data Online

### SOS2 Services for Water Data Online
https://github.com/awracms/awra_cms/blob/master/Training/Advanced/AlterObjectiveFunction_DownloadBoMdata.ipynb

The Bureau has implemented a Sensor Observation Service standard version 2 (SOS2) service to make publicly available the time series data it collects and manages under the Water Regulations 2008. The service provides data in WaterML2 format. The SOS2 standard was developed by the Open Geospatial Consortium (OGC) as part of the OGC Sensor Web Enablement framework. The WaterML2 standard was also developed by the OGC with support from the Bureau of Meteorology and CSIRO though the WIRADA research alliance.

In [ ]:
import sys
sys.path.append('../10_Scripts')

In [ ]:
import talk_to_wiski2 as W

In [ ]:
wcds = W.get_station_list("pH")

In [ ]:
import owslib.sos as sos

In [ ]:
service =sos.SensorObservationService('http://www.bom.gov.au/waterdata/services',version='2.0')

In [ ]:
service.contents

In [ ]:
# get set of stations (xml elements) that have 'Water Course Discharge'
import talk_to_wiski2 as W
wcds = W.get_station_list("Water Course Discharge")
#wcds = W.get_station_list("Water Course Discharge") # Get discharge volume (m3/sec) from wiski for gauging stations
#### find all datasets for a station "105001"
#stns = W.find_id_in_station_list('105001',wcds) 
#stns  #['105001B', '105001A']

### Plot the downloaded data for site '105001B' including quality codes

In [ ]:
import numpy as np
import pandas as pd
siteid='105001' # site id according to benchmarking dataset
whichsite=18.1 # find which of the set this corresponds to to get the true wrsids ID


# get the data 
a=pd.DataFrame(W.get_data(wrsids[whichsite],obs_type="Water Course Discharge",period=pd.date_range('1900-01-01','2017-10-23',freq='D'))) # Setup data to download from Wiski

# plot the data
%matplotlib inline
figsize=(12,7)
fig = plt.figure(figsize=figsize)
fig.suptitle(wrsids[whichsite] + ' Water Course Discharge [ML/day]')
ax1 = fig.add_subplot(3,1,1)
a1=pd.DataFrame(a['values'])
b=pd.DataFrame(qobs[str(int(ids[whichsite]))])
# Convert mm/day benchmarking data set to cumecs as per downloaded data
# Q(m^3/s) =Q (mm/day) *Catchment Area (m^2) / 1000mm/m * (24*3600second/day)   
# area is provided in km^2
# Q(m^3/s) =Q (mm/day) *Area (km^2)*1000*1000 /1000*24*3600 
#          =Q (mm/day) *Area (km^2)/24*3.6
#          =Q (mm/day) *Area (km^2)/86.4
b=b*areas[whichsite]/86.4
c=a1.join(b)
dt = pd.to_datetime(a.index, format='%Y-%m-%d')
ax1.plot_date(dt,c['values'],'-',label='Newly downloaded using webservices')
ax1.plot_date(dt,c[siteid],'-',label='Original benchmarking')
ax1.legend()
ax1.set_title('Streamflow [ML/day]')
ax1 = fig.add_subplot(3,1,2)
ax1.plot_date(dt,c,'-')  
plt.yscale('log')
ax1.set_title('Streamflow [ML/day] (log-y axis)')

# finaly qualaty codes
ax1 = fig.add_subplot(3,1,3)
ax1.plot(a['qualifier'],color='red')
ax1.set_title('BoM Quality codes'+'1034=A, 1114=B, 1134=C, 1164=E, 1234=F')
# see bom quality codes
# currently known issue in SOS with transllations of quality codes
#http://www.bom.gov.au/water/wdtf/documentation/schema-control-lists/quality-code.htm
#dictQualityCodes='1034=A, 1114=B, 1134=C, 1164=E, 1234=F'    
 

Attempt 2 code from:

https://github.com/peterataylor/bom-water-data-service-example/blob/master/bom-water-sos-example.ipynb

In [1]:
from owslib.sos import SensorObservationService
from owslib.swe.observation.sos200 import SOSGetObservationResponse
from IPython.core.display import HTML

service = SensorObservationService('http://www.bom.gov.au/waterdata/services',version='2.0.0')

ReadTimeout: HTTPConnectionPool(host='www.bom.gov.au', port=80): Read timed out. (read timeout=30)

In [ ]:
str =''
for prop in service.observed_properties:
    str += '<li>' + prop.replace('http://bom.gov.au/waterdata/services/parameters/','') + '</li>'
HTML(str)

In [ ]:
response = service.get_observation(featureOfInterest='http://bom.gov.au/waterdata/services/stations/181.1',
                                   offerings=['http://bom.gov.au/waterdata/services/tstypes/Pat4_PC_1'],
                                   observedProperties=['http://bom.gov.au/waterdata/services/parameters/Water Course Discharge'],
                                   eventTime='om:phenomenonTime,2016-01-01T00:00:00+10/2016-03-05T00:00:00+10', 
                                   headers={'User-Agent': 'Mozilla/5.0'})

basic example

In [ ]:
import owslib.sos as sos

In [ ]:
service =sos.SensorObservationService('http://www.bom.gov.au/waterdata/services',version='2.0')

In [ ]:
service.contents